In [1]:
import pmagpy
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import glob as glob
import shutil
import math
from pathlib import Path
%matplotlib inline

In [2]:
home = Path(os.getcwd())
data_folder = home/'data'
magic_folder = home/'magic'

In [3]:
def add_vgps(sites_df):
    # Outputs dataframe with columns: vgp_lat, vgp_lon, vgp_dp, vgp_dm
    data=sites_df[['dir_dec','dir_inc','dir_alpha95','lat','lon']].to_numpy()
    vgps=np.array(pmag.dia_vgp(data)) 
    vgps=vgps.transpose()
    return(pd.DataFrame(vgps,columns=['vgp_lon','vgp_lat','vgp_dp','vgp_dm'],index=sites_df.index))

def fisher_avg(samples_df):
    # Outputs dataframe with columns: dir_dec, dir_inc, dir_k, dir_alpha95, dir_n_sites
    # k and a95 are determined by parametric bootstrap
    data=samples_df[['dir_dec','dir_inc']].dropna()
    if (len(data)==1):
        data['dir_n_sites']=1
        data['result_type']='i'
        return(data)
    else:
        result=pd.DataFrame([pmag.dir_df_fisher_mean(data)]).rename(columns={'alpha95':'dir_alpha95',
                                  'dec':'dir_dec',
                                  'inc':'dir_inc',
                                  'k':'dir_k',
                                  'n':'dir_n_sites'})
        result['result_type']='a'
        return(result)

## Letts et al. 2009

In [4]:
## Letts et al (2009)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/16/2024): 
#  
#
#  Region: sites.location (Note: there are also 'locations' for each )
#  DecLong: sites.lon
#  DecLat: sites.lat
#  n (col. L): sites.dir_n_total_samples
#  Full Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  GDec: sites.dir_dec
#  GInc: sites.dir_inc
#  Strike: sites.bed_dip_direction (=(strike+90.)%360.)
#  Dip: sites.bed_dip_direction
#  n (col. W): sites.dir_n_samples
#  a95 (col. X): sites.dir_alpha95
#  kappa: sites.dir_k
#  Lithology: sites.lithologies
#  
# Assumptions:
#
#  sites.formation = "Bushveld Complex"
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1111/j.1365-246X.2009.04346.x"
#  sites.age = 2054
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if n (col. W) is not nan, otherwise "b"

letts_data = pd.read_excel(data_folder/'Letts_Bushveld.xlsx',sheet_name='Sheet1')
#display(letts_data)

In [5]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=letts_data['Full Site'].astype('string')
#sites_df['location']=letts_data.apply(lambda x: "{}:{}".format(x['Region'],x['Site']),axis=1)
sites_df['location']=letts_data['Region']
sites_df['lat']=letts_data['DecLat']
sites_df['lon']=letts_data['DecLong']
sites_df['dir_n_total_samples']=letts_data['n']
sites_df['dir_n_samples']=letts_data['n.1']
sites_df['dir_dec']=letts_data['GDec']
sites_df['dir_inc']=letts_data['GInc']
sites_df['bed_dip']=letts_data['Dip']
sites_df['bed_dip_direction']=(letts_data['Strike']+90.)%360.
sites_df['dir_alpha95']=letts_data['a95.1']
sites_df['dir_k']=letts_data['kappa']
sites_df['lithologies']= letts_data['Lithology']
sites_df['formation']= "Bushveld Complex"
sites_df['geologic_classes']='Igneous:Intrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1111/j.1365-246X.2009.04346.x"
sites_df['geologic_types']="Layered Intrusion"
sites_df['age']= 2054
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= letts_data.apply(lambda x: 'b' if np.isnan(x['n.1']) else 'g',axis=1)
#display(sites_df)

In [6]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [7]:
# Compile locations dataframe
locations_df1=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df1['location']=sites_group.groups.keys()
locations_df1['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['location_type']='Region'
locations_df1['geologic_classes']='Igneous:Intrusive'
locations_df1['lithologies']=sites_group['lithologies'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['lat_s']=sites_df['lat'].min()
locations_df1['lat_n']=sites_df['lat'].max()
locations_df1['lon_e']=sites_df['lon'].max()
locations_df1['lon_w']=sites_df['lon'].min()
locations_df1['age']=2054
locations_df1['age_unit']="Ma"
locations_df1=pd.concat([locations_df1,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df2=pd.DataFrame([])
sites_group=sites_df.groupby('formation')
locations_df2['location']=sites_group.groups.keys()
locations_df2['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('formation',axis=1)
locations_df2['location_type']='Region'
locations_df2['geologic_classes']='Igneous:Intrusive'
locations_df2['lithologies']=sites_group['lithologies'].unique().apply(':'.join).reset_index().drop('formation',axis=1)
locations_df2['lat_s']=sites_df['lat'].min()
locations_df2['lat_n']=sites_df['lat'].max()
locations_df2['lon_e']=sites_df['lon'].max()
locations_df2['lon_w']=sites_df['lon'].min()
locations_df2['age']=2054
locations_df2['age_unit']="Ma"
locations_df2=pd.concat([locations_df2,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df = pd.concat([locations_df1, locations_df2],axis=0).reset_index(drop=True)
#display(locations_df)

In [8]:
# Compile contributions dataframe
contribution_df=pd.DataFrame({'reference':["10.1111/j.1365-246X.2009.04346.x"],'lab_names':["Not Specified"]},index=[0])
print(contribution_df)

                          reference      lab_names
0  10.1111/j.1365-246X.2009.04346.x  Not Specified


In [9]:
contribution_dicts=contribution_df.fillna('').to_dict('records')
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [10]:
pmag.magic_write(magic_folder/'letts2009_contribution.txt', contribution_dicts, 'contribution')
pmag.magic_write(str(magic_folder/'letts2009_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'letts2009_sites.txt', sites_dicts, 'sites')

1  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_contribution.txt
6  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_locations.txt
101  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/letts2009_sites.txt'))

## Kosterov & Perrin (1996)

In [11]:
## Kosterov & Perrin (1996)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Location: sites.description (this is kind of a kludge to add in the strat section as a column in sites.txt)
#  ID: sites.site (had to add this column to remove ambiguities in site numbers)
#  Lat: sites.lat
#  Long: sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Inc: sites.dir_inc
#  Dec: sites.dir_dec
#  k: sites.dir_k
#  a95: sites.dir_alpha95
#  GroupID: sites.location ()
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1016/0012-821X(96)00005-2"
#  sites.age = 180
#  sites.age_sigma = 5
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group (col. N) is not nan, otherwise "b"

kosterov1996_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Kosterov 1996',skiprows=4).dropna(how='all')
#display(kosterov1996_data)

In [87]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=kosterov1996_data['ID'].astype('string')
sites_df['location']=kosterov1996_data['GroupID'].astype('string')
sites_df['location']=sites_df['location'].fillna('N/A')
sites_df['description']=kosterov1996_data['Location']
sites_df['lat']=kosterov1996_data['Lat']
sites_df['lon']=kosterov1996_data['Long']
sites_df['dir_n_total_samples']=kosterov1996_data['N']
sites_df['dir_n_samples']=kosterov1996_data['n']
sites_df['dir_dec']=kosterov1996_data['Dec']
sites_df['dir_inc']=kosterov1996_data['Inc']
sites_df['dir_alpha95']=kosterov1996_data['a95']
sites_df['dir_k']=kosterov1996_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1016/0012-821X(96)00005-2"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 180
sites_df['age_sigma']= 5
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['location'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

,site,location,description,lat,lon,...,age,age_sigma,age_unit,dir_tilt_correction,result_quality
0,M-101,GM-28,Mafika,-29.07,28.38,...,180,5,Ma,0,g
1,M-100,GM-28,Mafika,-29.07,28.38,...,180,5,Ma,0,g
2,M-95,GM-27,Mafika,-29.07,28.38,...,180,5,Ma,0,g
3,M-93,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
4,M-91,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
5,M-89,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
6,M-88,GM-26,Mafika,-29.07,28.38,...,180,5,Ma,0,g
7,M-86,GM-25,Mafika,-29.07,28.38,...,180,5,Ma,0,g
8,M-84,GM-24,Mafika,-29.07,28.38,...,180,5,Ma,0,g
9,M-82,GM-24,Mafika,-29.07,28.38,...,180,5,Ma,0,g


In [88]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [93]:
# Compile locations dataframe
locations_df1=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df1['location']=sites_group.groups.keys()
locations_df1['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['location_type']='Outcrop'
locations_df1['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['lithologies']='Basalt'
locations_df1['description']=sites_group.first()['description'].reset_index(drop=True)
locations_df1['lat_s']=sites_df['lat'].min()
locations_df1['lat_n']=sites_df['lat'].max()
locations_df1['lon_e']=sites_df['lon'].max()
locations_df1['lon_w']=sites_df['lon'].min()
locations_df1['age']=180
locations_df1['age_sigma']=5
locations_df1['age_unit']="Ma"
locations_df1=pd.concat([locations_df1,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)


locations_df2=pd.DataFrame([])
#sites_group=sites_df.groupby('description')
sites_group=locations_df1.groupby('description')

locations_df2['location']=sites_group.groups.keys()
new_sites=sites_group.apply(lambda x:":".join(list(set(x['sites'].str.split(':').explode().tolist())))).reset_index()
new_sites.columns=['location','sites']

locations_df2=pd.merge(locations_df2,new_sites,on='location')
locations_df2['location_type']='Stratigraphic Section'
locations_df2['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('description',axis=1)
locations_df2['lithologies']='Basalt'
locations_df2['lat_s']=sites_df['lat'].min()
locations_df2['lat_n']=sites_df['lat'].max()
locations_df2['lon_e']=sites_df['lon'].max()
locations_df2['lon_w']=sites_df['lon'].min()
locations_df2['age']=180
locations_df2['age_sigma']=5
locations_df2['age_unit']="Ma"
locations_df2=pd.concat([locations_df2,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df = pd.concat([locations_df1, locations_df2],axis=0).reset_index(drop=True)
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

,location,sites,location_type,geologic_classes,lithologies,...,dir_inc,dir_n_sites,dir_k,dir_alpha95,result_type
0,GM-1,M-1,Outcrop,Igneous:Extrusive,Basalt,...,-60.700000,1,NaN,NaN,i
1,GM-10,M-35:M-33,Outcrop,Igneous:Extrusive,Basalt,...,-60.952676,2,14099.637456,2.103428,a
2,GM-11,M-37,Outcrop,Igneous:Extrusive,Basalt,...,-62.900000,1,NaN,NaN,i
3,GM-12,M-39,Outcrop,Igneous:Extrusive,Basalt,...,-58.400000,1,NaN,NaN,i
4,GM-13,M-41,Outcrop,Igneous:Extrusive,Basalt,...,-45.800000,1,NaN,NaN,i
5,GM-14,M-43,Outcrop,Igneous:Extrusive,Basalt,...,-54.800000,1,NaN,NaN,i
6,GM-15,M-45,Outcrop,Igneous:Extrusive,Basalt,...,-58.300000,1,NaN,NaN,i
7,GM-16,M-47,Outcrop,Igneous:Extrusive,Basalt,...,-52.200000,1,NaN,NaN,i
8,GM-17,M-51:M-49,Outcrop,Igneous:Extrusive,Basalt,...,-52.652694,2,11014.668670,2.379879,a
9,GM-18,M-53,Outcrop,Igneous:Extrusive,Basalt,...,-63.700000,1,NaN,NaN,i


In [94]:
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [95]:
pmag.magic_write(str(magic_folder/'kosterov1996_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'kosterov1996_sites.txt', sites_dicts, 'sites')

52  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\kosterov1996_locations.txt
73  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\kosterov1996_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/kosterov1996_sites.txt'))

## Moulin et al. 2011

In [78]:
## Moulin et al. 2011
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.description
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Slat: sites.lat
#  Slon: sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Ig: sites.dir_inc
#  Dg: sites.dir_dec
#  k: sites.dir_k
#  a95: sites.dir_alpha95
#  Dir. Group or Flow: sites.location
#  Age: sites.age
#  dAge: sites.age_sigma
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1029/2011JB008210"
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2011_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2011',skiprows=4,usecols="A:S").dropna(how='all')
#display(moulin2011_data)

In [80]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2011_data['Site'].astype('string')
sites_df['formation']=moulin2011_data['Formation'].astype('string')
sites_df['location']=moulin2011_data['Dir. Group or Flow'].astype('string')
sites_df['description']=moulin2011_data['Section']
sites_df['lat']=moulin2011_data['Slat']
sites_df['lon']=moulin2011_data['Slon']
sites_df['dir_n_total_samples']=moulin2011_data['N']
sites_df['dir_n_samples']=moulin2011_data['n']
sites_df['dir_dec']=moulin2011_data['Dg']
sites_df['dir_inc']=moulin2011_data['Ig']
sites_df['dir_alpha95']=moulin2011_data['a95']
sites_df['dir_k']=moulin2011_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1029/2011JB008210"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= moulin2011_data['Age']
sites_df['age_sigma']= moulin2011_data['dAge']
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['location'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

,site,formation,location,description,lat,...,age,age_sigma,age_unit,dir_tilt_correction,result_quality
0,NN02,Mafika Lisiu,DG29,Naude's Nek Section,-30.7610,...,179.2,1.8,Ma,0,g
1,NN03,Mafika Lisiu,DG29,Naude's Nek Section,-30.7612,...,179.2,1.8,Ma,0,g
2,NN04,Mafika Lisiu,DG29,Naude's Nek Section,-30.7612,...,179.2,1.8,Ma,0,g
3,NN05,Mafika Lisiu,DG28,Naude's Nek Section,-30.7603,...,179.2,1.8,Ma,0,g
4,NN06,Mafika Lisiu,DG28,Naude's Nek Section,-30.7601,...,179.2,1.8,Ma,0,g
5,NN07,Mafika Lisiu,DG28,Naude's Nek Section,-30.7592,...,179.2,1.8,Ma,0,g
6,NN08,Mafika Lisiu,DG28,Naude's Nek Section,-30.7589,...,179.2,1.8,Ma,0,g
7,NN09,Mafika Lisiu,DG28,Naude's Nek Section,-30.7588,...,179.2,1.8,Ma,0,g
8,NN10,Mafika Lisiu,DG28,Naude's Nek Section,-30.7590,...,179.2,1.8,Ma,0,g
9,NN11,Mafika Lisiu,DG28,Naude's Nek Section,-30.7597,...,179.2,1.8,Ma,0,g


In [81]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [84]:
# Compile locations dataframe
locations_df1=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df1['location']=sites_group.groups.keys()
locations_df1['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['description']=sites_group.first()['description'].reset_index(drop=True)
locations_df1['location_type']='Outcrop'
locations_df1['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df1['lithologies']='Basalt'
locations_df1['lat_s']=sites_df['lat'].min()
locations_df1['lat_n']=sites_df['lat'].max()
locations_df1['lon_e']=sites_df['lon'].max()
locations_df1['lon_w']=sites_df['lon'].min()
locations_df1['age_low']=sites_df['age'].min()
locations_df1['age_high']=sites_df['age'].max()
locations_df1['age_sigma']=sites_df['age_sigma'].max()
locations_df1['age_unit']="Ma"
locations_df1=pd.concat([locations_df1,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df2=pd.DataFrame([])
#sites_group=sites_df.groupby('description')
sites_group=locations_df1.groupby('description')

locations_df2['location']=sites_group.groups.keys()
new_sites=sites_group.apply(lambda x:":".join(list(set(x['sites'].str.split(':').explode().tolist())))).reset_index()
new_sites.columns=['location','sites']

locations_df2=pd.merge(locations_df2,new_sites,on='location')
locations_df2['location_type']='Stratigraphic Section'
locations_df2['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('description',axis=1)
locations_df2['lithologies']='Basalt'
locations_df2['lat_s']=sites_df['lat'].min()
locations_df2['lat_n']=sites_df['lat'].max()
locations_df2['lon_e']=sites_df['lon'].max()
locations_df2['lon_w']=sites_df['lon'].min()
locations_df2['age_low']=sites_df['age'].min()
locations_df2['age_high']=sites_df['age'].max()
locations_df2['age_sigma']=sites_df['age_sigma'].max()
locations_df2['age_unit']="Ma"
locations_df2=pd.concat([locations_df2,sites_group.apply(lambda x: fisher_avg(x)).reset_index()[['dir_dec','dir_inc','dir_n_sites','dir_k','dir_alpha95','result_type']]],
                        axis=1)

locations_df = pd.concat([locations_df1, locations_df2],axis=0).reset_index(drop=True)
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

,location,sites,description,location_type,geologic_classes,...,dir_inc,dir_n_sites,dir_k,dir_alpha95,result_type
0,10,ND54,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-44.700000,1,NaN,NaN,i
1,12,ND49,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-56.100000,1,NaN,NaN,i
2,14,NNPR,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-49.300000,1,NaN,NaN,i
3,18,NN35C,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-66.400000,1,NaN,NaN,i
4,19,NN35B,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-51.100000,1,NaN,NaN,i
5,20,NN35A,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-54.300000,1,NaN,NaN,i
6,21,NN34,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-59.200000,1,NaN,NaN,i
7,22,NN33,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-58.200000,1,NaN,NaN,i
8,26,NN19,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-55.200000,1,NaN,NaN,i
9,8,ND57,Naude's Nek Section,Outcrop,Igneous:Extrusive,...,-29.700000,1,NaN,NaN,i


In [85]:
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_df.fillna('').to_dict('records')

In [86]:
pmag.magic_write(str(magic_folder/'moulin2011_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2011_sites.txt', sites_dicts, 'sites')

30  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\moulin2011_locations.txt
86  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\moulin2011_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/moulin2011_sites.txt'))

## Moulin et al. 2012

In [ ]:
## Moulin et al. 2012
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.location
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Lat: sites.lat
#  Long: sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Inc: sites.dir_inc
#  Dec: sites.dir_dec
#  k: sites.dir_k
#  a95: sites.dir_alpha95
#  Dir. Group: sites.group
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1029/2011GC003910"
#  sites.age = 179.2
#  sites.age_sigma = 1.8
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2012_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2012',skiprows=3,usecols="A:L").dropna(how='all')
display(moulin2012_data)

In [ ]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2012_data['Site'].astype('string')
sites_df['groups']=moulin2012_data['Dir. Group'].astype('string')
sites_df['location']=moulin2012_data['Section']
sites_df['lat']=moulin2012_data['Lat']
sites_df['lon']=moulin2012_data['Long']
sites_df['dir_n_total_samples']=moulin2012_data['N']
sites_df['dir_n_samples']=moulin2012_data['n']
sites_df['dir_dec']=moulin2012_data['Dec']
sites_df['dir_inc']=moulin2012_data['Inc']
sites_df['dir_alpha95']=moulin2012_data['a95']
sites_df['dir_k']=moulin2012_data['k']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1029/2011GC003910"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 179.2
sites_df['age_sigma']= 1.8
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['groups'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

In [ ]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [ ]:
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['location_type']='Stratigraphic Section'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['lithologies']='Basalt'
locations_df['lat_s']=sites_df['lat'].min()
locations_df['lat_n']=sites_df['lat'].max()
locations_df['lon_e']=sites_df['lon'].max()
locations_df['lon_w']=sites_df['lon'].min()
locations_df['age']=179.2
locations_df['age_sigma']=1.8
locations_df['age_unit']="Ma"
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

In [ ]:
locations_dicts=locations_df.to_dict('records')
sites_dicts = sites_df.to_dict('records')

In [ ]:
pmag.magic_write(str(magic_folder/'moulin2012_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2012_sites.txt', sites_dicts, 'sites')

## Moulin et al. 2017

In [ ]:
## Moulin et al. 2017
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/17/2024): 
#  
#
#  Section: sites.location
#  Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  Slat (deg): sites.lat
#  Slon (deg): sites.lon
#  N: sites.dir_n_total_samples
#  n: sites.dir_n_samples
#  Ig (deg): sites.dir_inc
#  Dg (deg): sites.dir_dec
#  K: sites.dir_k
#  a95 (deg): sites.dir_alpha95
#  Directional Group or Single Flow: sites.group
#  
# Assumptions:
#
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1002/2016JB013354"
#  sites.age = 181.1
#  sites.age_sigma = 1.0
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0
#  sites.result_quality = "g" if group is not nan, otherwise "b"

moulin2017_data = pd.read_excel(data_folder/'KarooData.xlsx',sheet_name='Moulin 2017',skiprows=1,usecols="A:L").dropna(how='all')
display(moulin2017_data)

In [ ]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=moulin2017_data['Site'].astype('string')
sites_df['groups']=moulin2017_data['Directional Group or Single Flow'].astype('string')
sites_df['location']=moulin2017_data['Section']
sites_df['lat']=moulin2017_data['Slat (deg)']
sites_df['lon']=moulin2017_data['Slon (deg)']
sites_df['dir_n_total_samples']=moulin2017_data['N']
sites_df['dir_n_samples']=moulin2017_data['n']
sites_df['dir_dec']=moulin2017_data['Dg (deg)']
sites_df['dir_inc']=moulin2017_data['Ig (deg)']
sites_df['dir_alpha95']=moulin2017_data['a95 (deg)']
sites_df['dir_k']=moulin2017_data['K']
sites_df['lithologies']= 'Basalt'
sites_df['geologic_classes']='Igneous:Extrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1002/2016JB013354"
sites_df['geologic_types']="Lava Flow"
sites_df['age']= 181.1
sites_df['age_sigma']= 1.0
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']= 'g'
sites_df.loc[sites_df['groups'].isna(),'result_quality']='b'
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(sites_df)

In [ ]:
# Calculate and attach VGPs
vgps_df=add_vgps(sites_df)
sites_df = pd.concat([sites_df,vgps_df],axis=1)

In [ ]:
# Compile locations dataframe
locations_df=pd.DataFrame([])
sites_group=sites_df.groupby('location')
locations_df['location']=sites_group.groups.keys()
locations_df['sites']=sites_group['site'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['location_type']='Stratigraphic Section'
locations_df['geologic_classes']=sites_group['geologic_classes'].unique().apply(':'.join).reset_index().drop('location',axis=1)
locations_df['lithologies']='Basalt'
locations_df['lat_s']=sites_df['lat'].min()
locations_df['lat_n']=sites_df['lat'].max()
locations_df['lon_e']=sites_df['lon'].max()
locations_df['lon_w']=sites_df['lon'].min()
locations_df['age']=181.1
locations_df['age_sigma']=1.0
locations_df['age_unit']="Ma"
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(locations_df)

In [ ]:
locations_dicts=locations_df.to_dict('records')
sites_dicts = sites_df.to_dict('records')

In [ ]:
pmag.magic_write(str(magic_folder/'moulin2017_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'moulin2017_sites.txt', sites_dicts, 'sites')